    !pip install pandas
    !pip install numpy
    !pip install matplotlib
    !pip install seaborn
    !pip install nltk
    !pip install beautifulsoup4
    !pip install requests
    !pip install contractions
    !pip install unidecode
    !pip install autocorrect
    !pip install scikit-learn
    
    Import Any libraries you need to work upon

##### Importing the Required Libraries!

In [27]:
# Libraries used for handling Dataframes
import pandas as pd, numpy as np

# Libraries used for visualizing Graphs
import matplotlib.pyplot as plt
import seaborn as sns

# Libraries used for NLP and Regular Expressions
import nltk, re, requests, os
# Libraries used for handling Dataframes
import warnings

# Warnings as per our required!
warnings.filterwarnings('ignore')

    Reading the dataset and finding the use ful info :-

In [15]:
soup_df = pd.read_excel('Input.xlsx')
soup_df.head()

,URL_ID,URL
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...
3,bctech2014,https://insights.blackcoffer.com/effective-man...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...


In [16]:
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

In [17]:
def extractor(url) :
    response = requests.get(url)
    if response.status_code != 200 :
        return None
    sp = soup(response.content, 'html.parser')
    title = sp.find('h1')
    if title: title_text = title.get_text(strip=True)
    else: title_text = "No Title Found"

    article = sp.find('article')
    if not article: article = soup.find('div', class_='article-content')

    if article: article_text = article.get_text(strip=True)
    else: article_text = "No Content Found"
        
    return title_text + "\n\n" + article_text

In [18]:
soup_df['Text'] = soup_df['URL'].apply(extractor)
soup_df.head()

,URL_ID,URL,Text
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...


    Let's save the file so that to work upon later and easily,..

In [19]:
soup_df.to_csv('Input_Text.csv', index = False)

    Reading the Article to work More Upon.

In [20]:
soup_df = pd.read_csv('Input_Text.csv')
soup_df

,URL_ID,URL,Text
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML and AI-based insurance premium model to pre...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration: Interactive Brokers A...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration and User-Friendly I...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management of Social Media Data Extr...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface for M...
...,...,...,...
142,bctech2153,https://insights.blackcoffer.com/population-an...,Population and Community Survey of America\n\n...
143,bctech2154,https://insights.blackcoffer.com/google-lsa-ap...,Google LSA API Data Automation and Dashboardin...
144,bctech2155,https://insights.blackcoffer.com/healthcare-da...,Healthcare Data Analysis\n\nHomeOur Success St...
145,bctech2156,https://insights.blackcoffer.com/budget-sales-...,"Budget, Sales KPI Dashboard using Power BI\n\n..."


In [26]:
soup_df.iloc[0,2]

'ML and AI-based insurance premium model to predict premium to be charged by the insurance company\n\nHomeOur Success StoriesML and AI-based insurance premium model to predict premium to be charged...Our Success StoriesBanking, Financials, Securities, and InsuranceML and AI-based insurance premium model to predict premium to be charged by the insurance companyByAjay Bidyarthy-January 7, 20248121Client BackgroundClient:A leading insurance firm worldwideIndustry Type:BFSIProducts & Services:InsuranceOrganization Size:10000+The ProblemThe insurance industry, particularly in the context of providing coverage to Public Company Directors against Insider Trading public lawsuits, faces a significant challenge in accurately determining insurance premiums. Traditional methods of premium calculation may lack precision, and there is a growing need for more sophisticated and data-driven approaches. The integration of Artificial Intelligence (AI) and Machine Learning (ML) models in predicting insura

In [34]:
def multi_join(file, var) :
    for i in file:
        if os.path.exists(i):
            with open(i, 'r') as f:
                var.update(f.read().splitlines())
                
def single_join(file, var) :
    if os.path.exists(file):
        with open(file, 'r') as f:
            var.update(f.read().splitlines())

In [35]:
stop_words_files = [
    './StopWords/StopWords_Currencies.txt',
    './StopWords/StopWords_Auditor.txt',
    './StopWords/StopWords_DatesandNumbers.txt',
    './StopWords/StopWords_Generic.txt',
    './StopWords/StopWords_GenericLong.txt',
    './StopWords/StopWords_Geographic.txt',
    './StopWords/StopWords_Names.txt'
]
positive_words_path = './MasterDictionary/positive-words.txt'
negative_words_path = './MasterDictionary/negative-words.txt'

stop_words = set()
positive_words = set()
negative_words = set()

multi_join(stop_words_files, stop_words)
single_join(positive_words_path, positive_words)
single_join(negative_words_path, negative_words)

print(f"Loaded {len(stop_words)} stop words.")
print(f"Loaded {len(positive_words)} positive words.")
print(f"Loaded {len(negative_words)} negative words.")

Loaded 12919 stop words.
Loaded 2006 positive words.
Loaded 4783 negative words.


    Some Cleaning Functions to make the Text and the values Clean.
    
    And UseFul libraries for this.

In [97]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, cmudict
from string import punctuation
import contractions
from unidecode import unidecode
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from autocorrect import Speller

In [41]:
def remove_stop_words(text, stop_words):
    words = word_tokenize(text)
    
    filtered_words = []
    
    for word in words:
        if word.lower() not in stop_words:
            filtered_words.append(word)
    
    return ' '.join(filtered_words)

In [44]:
cleaned_texts = []

for index, row in soup_df.iterrows():
    cleaned_text = remove_stop_words(row['Text'], stop_words)
    cleaned_texts.append(cleaned_text)

cleaned_texts

['ML AI-based insurance premium model predict premium charged insurance company HomeOur Success StoriesML AI-based insurance premium model predict premium charged ... Success StoriesBanking , Financials , Securities , InsuranceML AI-based insurance premium model predict premium charged insurance companyByAjay Bidyarthy-January 7 , 20248121Client BackgroundClient : leading insurance firm worldwideIndustry Type : BFSIProducts & Services : InsuranceOrganization Size:10000+The ProblemThe insurance industry , context providing coverage Public Company Directors Insider Trading public lawsuits , faces significant challenge accurately determining insurance premiums . Traditional methods premium calculation lack precision , growing sophisticated data-driven approaches . integration Artificial Intelligence ( AI ) Machine Learning ( ML ) models predicting insurance premiums specialized coverage essential enhance accuracy , fairness , responsiveness adapting evolving risk factors.The problem hand 

In [53]:
for i in range(0, 5) :
    print(f"For the Actual Info : Data - {i}")
    print(f" Length of the Words Before removing Stopwords : {len(soup_df['Text'][i])}.")
    print(f" Length of the Words After removing Stopwords : {len(cleaned_texts[i])}.")

For the Actual Info : Data - 0
 Length of the Words Before removing Stopwords : 20601.
 Length of the Words After removing Stopwords : 17897.
For the Actual Info : Data - 1
 Length of the Words Before removing Stopwords : 5263.
 Length of the Words After removing Stopwords : 4725.
For the Actual Info : Data - 2
 Length of the Words Before removing Stopwords : 5866.
 Length of the Words After removing Stopwords : 5297.
For the Actual Info : Data - 3
 Length of the Words Before removing Stopwords : 5017.
 Length of the Words After removing Stopwords : 4522.
For the Actual Info : Data - 4
 Length of the Words Before removing Stopwords : 5536.
 Length of the Words After removing Stopwords : 4825.


In [54]:
soup_df['Text'] = cleaned_texts
soup_df.head()

,URL_ID,URL,Text
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML AI-based insurance premium model predict pr...
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,Streamlined Integration : Interactive Brokers ...
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,Efficient Data Integration User-Friendly Inter...
3,bctech2014,https://insights.blackcoffer.com/effective-man...,Effective Management Social Media Data Extract...
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,Streamlined Trading Operations Interface MetaT...


In [57]:
len(soup_df['Text'][0])

17897

    Sussecfully removed the Stopwords now moving to the Next.

#### 1	Sentimental Analysis

In [69]:
def calculate_scores(text, positive_words, negative_words):
    words = word_tokenize(text)
    
    positive_score = 0
    negative_score = 0
    
    # Loop through each word to calculate positive and negative scores
    for word in words:
        if word.lower() in positive_words:
            positive_score += 1  # Increment for positive words
        elif word.lower() in negative_words:
            negative_score += 1  # Increment for negative words

    # Calculate Polarity Score
    total_score = positive_score + negative_score
    if total_score == 0:
        polarity_score = 0  # Handle division by zero
    else:
        polarity_score = (positive_score - negative_score) / (total_score + 0.000001)
    
    # Calculate Subjectivity Score
    total_words_after_cleaning = len(words)
    if total_words_after_cleaning == 0:
        subjectivity_score = 0  # Handle division by zero
    else:
        subjectivity_score = (positive_score + negative_score) / (total_words_after_cleaning + 0.000001)

    return positive_score, negative_score, polarity_score, subjectivity_score


In [71]:
positive_words = set(positive_words)
negative_words = set(negative_words)

positive_scores = []
negative_scores = []
polarity_scores = []
subjectivity_scores = []

for index, row in soup_df.iterrows():
    pos_score, neg_score, pol_score, subj_score = calculate_scores(row['Text'], positive_words, negative_words)
    
    positive_scores.append(pos_score)
    negative_scores.append(neg_score)
    polarity_scores.append(pol_score)
    subjectivity_scores.append(subj_score)

In [73]:
soup_df['positive_score'] = positive_scores
soup_df['negative_score'] = negative_scores
soup_df['polarity_score'] = polarity_scores
soup_df['subjectivity_score'] = subjectivity_scores

In [75]:
soup_df.head(1)

,URL_ID,URL,Text,positive_score,negative_score,polarity_score,subjectivity_score
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML AI-based insurance premium model predict pr...,110,38,0.486486,0.070915


#### 2	Analysis of Readability

In [80]:
d = cmudict.dict()

def syllable_count(word):
    syllables = d.get(word.lower())
    if syllables:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in syllables])
    else:
        return 0

def calculate_readability_metrics(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    
    num_sentences = len(sentences)
    num_words = len(words)
    
    # Calculate Average Sentence Length
    average_sentence_length = num_words / num_sentences
    
    # Count complex words
    complex_words = [word for word in words if syllable_count(word) >= 3]
    num_complex_words = len(complex_words)

    # Calculate Percentage of Complex Words
    percentage_complex_words = num_complex_words / num_words
    
    # Calculate Fog Index
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)

    return average_sentence_length, percentage_complex_words, fog_index

In [81]:
average_sentence_lengths = []
percentage_complex_words = []
fog_indexes = []

for index, row in soup_df.iterrows():
    avg_sentence_length, perc_complex_words, fog_index = calculate_readability_metrics(row['Text'])
    
    average_sentence_lengths.append(avg_sentence_length)
    percentage_complex_words.append(perc_complex_words)
    fog_indexes.append(fog_index)

In [82]:
soup_df['average_sentence_length'] = average_sentence_lengths
soup_df['percentage_complex_words'] = percentage_complex_words
soup_df['fog_index'] = fog_indexes

In [85]:
soup_df.head(1)

,URL_ID,URL,Text,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words,fog_index
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML AI-based insurance premium model predict pr...,110,38,0.486486,0.070915,67.322581,0.356013,27.071438


#### 3	Average Number of Words Per Sentence

In [89]:
def avg_word(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)
    
    num_sentences = len(sentences)
    num_words = len(words)
    
    # Calculate Average Number of Words Per Sentence
    average_words_per_sentence = num_words / num_sentences if num_sentences > 0 else 0

    return average_words_per_sentence

In [93]:
soup_df['average_words_per_sentence'] = soup_df['Text'].apply(calculate_average_words_per_sentence)
soup_df.head(1)

,URL_ID,URL,Text,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words,fog_index,average_words_per_sentence
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML AI-based insurance premium model predict pr...,110,38,0.486486,0.070915,67.322581,0.356013,27.071438,67.322581


    Complex Word Count, 
    Word Count, 
    Syllable Count Per Word, 
    Personal Pronouns, 
    Average Word Length

In [98]:
def complex_word_count(text):
    words = word_tokenize(text)
    count = 0
    for word in words:
        if syllable_count(word) > 2:
            count += 1
    return count

def word_count(text, stop_words):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(text)
    # Remove stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return len(filtered_words)

def syllable_count_per_word(text):
    words = word_tokenize(text)
    syllable_counts = {}
    
    for word in words:
        count = syllable_count(word)
        syllable_counts[word] = count
        
    return syllable_counts

def personal_pronoun_count(text):
    pronouns = r'\b(I|we|my|ours|us)\b'
    return len(re.findall(pronouns, text, flags=re.IGNORECASE))


def average_word_length(text):
    words = word_tokenize(text)
    total_chars = sum(len(word) for word in words)
    num_words = len(words)
    return total_chars / num_words if num_words > 0 else 0


In [99]:
soup_df['complex_word_count'] = soup_df['Text'].apply(complex_word_count)
soup_df['word_count'] = soup_df['Text'].apply(lambda x: word_count(x, stop_words))
soup_df['syllable_count_per_word'] = soup_df['Text'].apply(lambda x: word_count(x, stop_words))
soup_df['personal_pronoun_count'] = soup_df['Text'].apply(personal_pronoun_count)
soup_df['average_word_length'] = soup_df['Text'].apply(average_word_length)

In [102]:
soup_df.head(1)

,URL_ID,URL,Text,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words,fog_index,average_words_per_sentence,complex_word_count,word_count,syllable_count_per_word,personal_pronoun_count,average_word_length
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,ML AI-based insurance premium model predict pr...,110,38,0.486486,0.070915,67.322581,0.356013,27.071438,67.322581,743,1762,1762,0,7.575946


In [103]:
soup_df = soup_df.drop(columns = "Text")
soup_df.head()

,URL_ID,URL,positive_score,negative_score,polarity_score,subjectivity_score,average_sentence_length,percentage_complex_words,fog_index,average_words_per_sentence,complex_word_count,word_count,syllable_count_per_word,personal_pronoun_count,average_word_length
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,110,38,0.486486,0.070915,67.322581,0.356013,27.071438,67.322581,743,1762,1762,0,7.575946
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,22,4,0.692308,0.055437,156.333333,0.240938,62.629709,156.333333,113,421,421,0,9.076759
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,26,8,0.529412,0.055556,204.000000,0.251634,81.700654,204.000000,154,516,516,0,7.656863
3,bctech2014,https://insights.blackcoffer.com/effective-man...,16,3,0.684210,0.041215,153.666667,0.232104,61.559508,153.666667,107,416,416,0,8.811280
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,16,3,0.684210,0.032313,147.000000,0.255102,58.902041,147.000000,150,478,478,0,7.207483


In [105]:
output_df = pd.read_excel('Output Data Structure.xlsx')
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bctech2014,https://insights.blackcoffer.com/effective-man...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
reg = pd.read_excel('Output Data Structure.xlsx')
reg.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bctech2014,https://insights.blackcoffer.com/effective-man...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
soup_list = soup_df.columns
soup_list

Index(['URL_ID', 'URL', 'positive_score', 'negative_score', 'polarity_score',
       'subjectivity_score', 'average_sentence_length',
       'percentage_complex_words', 'fog_index', 'average_words_per_sentence',
       'complex_word_count', 'word_count', 'syllable_count_per_word',
       'personal_pronoun_count', 'average_word_length'],
      dtype='object')

In [113]:
output_list = output_df.columns
output_list

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [116]:
for i in range(len(output_list)):
    if output_list[i] not in ['URL_ID', 'URL']:
        output_df[output_list[i]] = soup_df[soup_list[i]]

In [118]:
output_df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,bctech2011,https://insights.blackcoffer.com/ml-and-ai-bas...,110,38,0.486486,0.070915,67.322581,0.356013,27.071438,67.322581,743,1762,1762,0,7.575946
1,bctech2012,https://insights.blackcoffer.com/streamlined-i...,22,4,0.692308,0.055437,156.333333,0.240938,62.629709,156.333333,113,421,421,0,9.076759
2,bctech2013,https://insights.blackcoffer.com/efficient-dat...,26,8,0.529412,0.055556,204.000000,0.251634,81.700654,204.000000,154,516,516,0,7.656863
3,bctech2014,https://insights.blackcoffer.com/effective-man...,16,3,0.684210,0.041215,153.666667,0.232104,61.559508,153.666667,107,416,416,0,8.811280
4,bctech2015,https://insights.blackcoffer.com/streamlined-t...,16,3,0.684210,0.032313,147.000000,0.255102,58.902041,147.000000,150,478,478,0,7.207483


In [119]:
output_df.to_excel('Output Data Structure Filled.xlsx', index = False)

# Sooja Parth Samay aayega jab tu bhi earn karegi 
# bas tab sone ko nhi milega Soo
# Good Night Parth

    Delete kardena